In [1]:
%pip install -U langchain langchain-community langchain-core sentence_transformers langchain-openai python-dotenv beautifulsoup4 langchain-chroma langchain-groq langchain-ollama FlagEmbedding peft gradio

  Attempting uninstall: langchain-groq
    Found existing installation: langchain-groq 0.1.9
    Uninstalling langchain-groq-0.1.9:
      Successfully uninstalled langchain-groq-0.1.9
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# https://mer.vin/2024/02/ollama-embedding/ - original source code
import sys
print(f"Python interpreter: {sys.executable}") # getting python interpreter

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community import embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings


# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # change model and embedding #c1
from langchain_community.embeddings import OllamaEmbeddings

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

from urllib.parse import urlparse, unquote

import os
from dotenv import load_dotenv
from pathlib import Path  


import shutil
import requests
import re
from langchain.docstore.document import Document 

Python interpreter: c:\Users\User-PC\Documents\Overseas stuff\Edinburgh\Year 3\coding projects\Ollama RAG\myenv\Scripts\python.exe


In [3]:
load_dotenv()
persistent_directory = "C:\\Users\\User-PC\\Documents\\Overseas stuff\\Edinburgh\\Year 3\\coding projects\\Ollama RAG\\db\\chromadb"


def extract_page_name(docs):
    for item in docs: 
        full_url = item.metadata.get("source")
        parsed_url = urlparse(full_url)
        page_name = unquote(parsed_url.path.split('/')[-1])

        item.metadata["id"] = page_name
        print(item.metadata["id"])
        

# claude's suggestion
# def scrape_jina_ai_1(url: str) -> Document:  
#     response = requests.get("https://r.jina.ai/" + url) 
#     content = response.text
    
#     content = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', content)
    
#     # Remove markdown-style links [text](url)
#     content = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', content)
    
#     # Remove HTML link tags
#     content = re.sub(r'<a\s+(?:[^>]*?\s+)?href="([^"]*)".*?>(.*?)<\/a>', r'\2', content)
    
#     return Document(page_content=content, metadataz={"source": url}) 

def scrape_jina_ai_2(url: str) -> Document:  
    response = requests.get("https://r.jina.ai/" + url) 
    content = response.text
    
    # remove urls
    content = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', content)
    
    return Document(page_content=content, metadata={"source": url}) 

    

In [5]:
# TEST
# jina_doc1 = scrape_jina_ai_1("https://frackinuniverse.miraheze.org/wiki/Stars")
jina_doc2 = scrape_jina_ai_2("https://frackinuniverse.miraheze.org/wiki/Stars")

# print(jina_doc1.page_content)
print("\n======== DOC 2 vv =========\n")
print(jina_doc2.page_content)


======== DOC 2 vv =========

Title: Stars - Frackin' Universe Wiki

URL Source: 

Published Time: 2023-08-30T05:28:59Z

Markdown Content:
Frackin' Universe introduces five new star types on top of the [vanilla star types]( Each star system can contain a variety of [planets]( "Planets") and has a specific threat level ([difficulty]( for space encounters.

General Characteristics\[[edit]( "Edit section: General Characteristics")\]
------------------------------------------------------------------------------------------------------------------------------------------------

Stars added in the Frackin' Universe have five regions where the planets are placed, and each region can have one or more orbitals. In each orbital, there is a 70% chance of placing a body. There is a ~86% chance of that body being a terrestrial planet, ~9% chance of it being a [Gas Giant]( "Gas Giant"), and ~5% chance of it being a field of [Asteroids]( "Asteroids").

Like vanilla, each terrestrial planet spawns up 

In [6]:
# embedding = OpenAIEmbeddings( model = "text-embedding-3-small") # using openAI embedding

model_name = "BAAI/bge-large-en-v1.5"           # using bge as embedding
encode_kwargs = {'normalize_embeddings': True}  # set True to compute cosine similarity
embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
) 

# from LLM for devs: https://github.com/trancethehuman/ai-workshop-code/blob/main/Web_scraping_for_LLM_in_2024.ipynb



force_rebuild = True

# if not os.path.exists(persistent_directory):    # setting up vector store
# if True:    # setting up vector store
if force_rebuild:  
    # Delete the existing vector store directory  
    if os.path.exists(persistent_directory):  
        print(f"Deleting existing vector store directory: {persistent_directory}")  
        shutil.rmtree(persistent_directory)  
        
    print("Initializing vector store...")

    urls = [
        "https://frackinuniverse.miraheze.org/wiki/Main_Page",
        "https://frackinuniverse.miraheze.org/wiki/Getting_Started", 
        "https://frackinuniverse.miraheze.org/wiki/Personal_Tricorder",
        "https://frackinuniverse.miraheze.org/wiki/The_Player",
        
        "https://frackinuniverse.miraheze.org/wiki/Stars",
        "https://frackinuniverse.miraheze.org/wiki/Crafting",
        "https://frackinuniverse.miraheze.org/wiki/Combat",
        "https://frackinuniverse.miraheze.org/wiki/Weapons",
        "https://frackinuniverse.miraheze.org/wiki/Planets",
        "https://frackinuniverse.miraheze.org/wiki/Biomes"
    ]
    # loading urls 
    # docs = [WebBaseLoader(url).load() for url in urls]
    # docs_list = [item for sublist in docs for item in sublist]
    
    docs_list = [scrape_jina_ai_2(url) for url in urls] # scrape using Jina AI's reader
    print("BIOMES PAGE >>> ")
    print(docs_list[-1].page_content+"\n")

    # set page name in metadata
    extract_page_name(docs_list)

    # split document into chunks
    # TODO: experiment with chunk size
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
    doc_splits = text_splitter.split_documents(docs_list)

    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(doc_splits)}")
    print(f"Sample chunk\n{doc_splits[0]}\n")


    # Convert documents to Embeddings and store them
    print("\n--- Creating vector store ---")
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        # collection_name="rag-openai",
        persist_directory=persistent_directory,
        embedding = embedding,
    )
    print("\n--- Finished creating vector store ---")
else: 
    print("Vector store already exists. No need to initialize")
    
    




c:\Users\User-PC\Documents\Overseas stuff\Edinburgh\Year 3\coding projects\Ollama RAG\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Deleting existing vector store directory: C:\Users\User-PC\Documents\Overseas stuff\Edinburgh\Year 3\coding projects\Ollama RAG\db\chromadb
Initializing vector store...
BIOMES PAGE >>> 
Title: Biomes - Frackin' Universe Wiki

URL Source: 

Published Time: 2023-11-22T04:36:09Z

Markdown Content:
From Frackin' Universe Wiki

[Jump to navigation](#mw-head) [Jump to search](#searchInput)

FU adds a plethora of planetary and minor biomes, but also enhances all [vanilla biomes]( Some biomes require specialized gear to safely traverse due to [hazards]( "Hazard (page does not exist)") not found in vanilla Starbound.

### Major Planetary Biomes\[[edit]( "Edit section: Major Planetary Biomes")\]

Image

Biome Name

[Star Types]( "Star Types (page does not exist)")

[Threat Levels]( "Threat Levels (page does not exist)")

Protection

Reason to visit

[![](

**[Aether Sea]( "Aether Sea")**

 [![]( "Black Star") [Black Star]( "Black Star")

5-8

 [![]( "Acid Immunity") [Acid Immunity]( "Acid Immuni

In [17]:
vectorstore = Chroma(persist_directory=persistent_directory,
            embedding_function=embedding)
# retriever = vectorstore.as_retriever(search_kwargs={"k":6})  # initialize retriever
retriever = vectorstore.as_retriever(search_kwargs={"k":10})  # initialize retriever

from FlagEmbedding import FlagReranker

# Add this function after your existing imports and before the main code
def rerank_and_select(query, documents, top_k=3):
    reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)
    
    # Prepare passages for reranking
    passages = [doc.page_content for doc in documents]
    
    # rerank_scores is a list of floats, e.g., [0.95, 0.87, 0.76, ...]
    rerank_scores = reranker.compute_score([[query, p] for p in passages])
    
    # Sort documents based on scores | ranked_results gets a sorted list of tuples (if an iterator of sets were passed, then sorted() returns a sorted list of sets), where each tuple is (float, Document)
    ranked_results = sorted(zip(rerank_scores, documents), key=lambda x: x[0], reverse=True)
    # INSIDES --> result of zip is an iterator of tuples, where each tuple is (float, Document) | iterators are 'consumed' only ONCE
    #             key : the tuples based on the rerank_score, the 1st part element of the tuple
    #             reverse: we want the largest score to be placed first (descending)
    
    # Select top k results
    top_results = ranked_results[:top_k]
    
    # Extract the original documents from the top results
    selected_docs = [result[1] for result in top_results]
    
    return selected_docs

def retrieve_and_format(query):
    relevant_docs = retriever.invoke(query)
    
    reranked_docs = rerank_and_select(query,relevant_docs)
    
    print('\n>>>> SOURCES <<<<< :')
    print([doc.metadata.get("id") for doc in reranked_docs])
    
    print_page_contents(reranked_docs)
    
    return "\n\n".join([doc.page_content for doc in reranked_docs])

def print_page_contents(docs):
    i=1
    for doc in docs:
        print(f"======= Doc {i} =======")
        print(doc.page_content)
        # print(doc.page_content[:100])
        i+=1
        


model_local = ChatOpenAI(   
    model="gpt-4o-mini",
    temperature = 0.1
    )

# model_local = ChatGroq(
#         api_key=os.getenv("GROQ_API_KEY"),
#         model="llama3-8b-8192", # this > llama 3.1 8b-instant (performance)
#         # model="llama-3.1-8b-instant",
#         # model="llama-3.1-70b-versatile",
#         temperature=0,
#         )

# model_local = ChatOllama(
#                 model="llama3.1",
#                 temperature=0.5)



C:\Users\User-PC\AppData\Local\Temp\ipykernel_20924\2342592546.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(persist_directory=persistent_directory,


In [18]:

# question = "List out all one-handed melee weapons"
question = "what are the features of the personal tricorder?"
exit_keyword = "exit"

# 4. After RAG
rag_template = """
                Role: You're the best Starbound Frackin' Universe enthusiast who's really pedantic and likes details but with concise and to-the-point sentences.
                Task:  
                Focus on the MAIN contents of the body, ignoring the periphery i.e., navigation bars, headers, footers, social media etc. 
                Give me the answer straight-away without any conversational preamble.
                Answer the question based only on the following context. If the information is not in the context, it is CRITICAL that you say you don't have the information
                :
                {context}
                Question: {question}
                """
                
llama_template = """
                <|begin_of_text|><|start_header_id|>system<|end_header_id|>
                You are a pedantic but knowledgeable, efficient and direct AI assistant for Frackin' Universe Website. Provide concise answers focusing on key information. Offer tactful suggestions to solve the user's question. 
                Answer the *question* based only on the following *context*. If the information is not in the *context*, say that you don't have the informmation.
                Focus on the MAIN contents of the given webpage in the *context*, ignoring the periphery of the website i.e., navigation bars, headers, footers, social media etc. 
                It is **CRITICAL** that you thoroughly digest the given *context* to answer the user's *question*.
                Context: {context}
                <|eot_id|>
                <|start_header_id|>user<|end_header_id|>
                {question}
                <|eot_id|>
                <|start_header_id|>assistant<|end_header_id|>
                """
# rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_prompt = ChatPromptTemplate.from_template(llama_template)
rag_chain = (
    {"context": RunnableLambda(retrieve_and_format), "question": RunnablePassthrough()}
    | rag_prompt
    | model_local
    | StrOutputParser()
)
result = rag_chain.invoke(question)

print("********ANSWER********")
print(result)

# while True:  # to have an ongoing conversation
    # user_question = input("\nHuman: ").strip()

    # if user_question.lower() == exit_keyword.lower():
    #     print("Exiting the program. Goodbye!")
    #     break

    # if user_question:
    #     print("\nProcessing your question...\n")
    #     nomic_response = rag_chain.invoke(user_question)
    #     openai_response = rag_chain_openai.invoke(user_question)
    #     print("====== Nomic Answer======:\n ",nomic_response)
    #     print("\n====== OpenAI Answer ======\n",openai_response)
    # else:
    #     print("Please enter a valid question")

# loader = PyPDFLoader("Ollama.pdf")
# doc_splits = loader.load_and_split()



>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Personal_Tricorder']
======= Doc 1 =======
Title: Personal Tricorder - Frackin' Universe Wiki

URL Source: 

Published Time: 2024-06-15T21:21:06Z

Markdown Content:
[![](

[![](

Personal Tricorder

Primary: Check StatusSecondary: Quest Menu Shift + Primary: Research Tree Shift + Secondary: Configure MM

Category

Console

Rarity

essential

Price

44

Stack size

1

Two-handed?

Yes

Tags

[tool]( "Category:ItemTag:tool")

ID

statustablet

**Personal Tricorder** is an incredibly important tool. New players always start with it. If you lose it, you can buy it at [Lost and Found]( "Lost and Found") and some other shops.

Features\[[edit]( "Edit section: Features")\]
-------------------------------------------------------------------------------------------------------------------------------

### Shift + left-click\[[edit]( "Edit section: Shift + left-click")\]

**Holding Shift and left-clicking** opens [Research]( "Re

In [19]:
tricorder_2 = "where can i find this tricorder if i happen to lose it?"
result_2 = rag_chain.invoke(tricorder_2)
print(result_2)


>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Main_Page']
======= Doc 1 =======
Title: Personal Tricorder - Frackin' Universe Wiki

URL Source: 

Published Time: 2024-06-15T21:21:06Z

Markdown Content:
[![](

[![](

Personal Tricorder

Primary: Check StatusSecondary: Quest Menu Shift + Primary: Research Tree Shift + Secondary: Configure MM

Category

Console

Rarity

essential

Price

44

Stack size

1

Two-handed?

Yes

Tags

[tool]( "Category:ItemTag:tool")

ID

statustablet

**Personal Tricorder** is an incredibly important tool. New players always start with it. If you lose it, you can buy it at [Lost and Found]( "Lost and Found") and some other shops.

Features\[[edit]( "Edit section: Features")\]
-------------------------------------------------------------------------------------------------------------------------------

### Shift + left-click\[[edit]( "Edit section: Shift + left-click")\]

**Holding Shift and left-clicking** opens [Research]( "Research") 

In [20]:
tricorder_3 = "Can i craft a personal tricorder without having to use my pixels?"
result_3 = rag_chain.invoke(tricorder_3)
print(result_3)


>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Main_Page']
======= Doc 1 =======
[![](

How it looks (main tab of Tricorder).

[![](

The Quickbar (if you have either [Stardust Core Light]( or [Stardust Core]( mod) gives fast access to its primary and secondary features, but not its MM Config features.

How to obtain
-------------

#### [Lorewalker's Shop]( "Lorewalker's Shop")

*   **200x** [Pixels]( "Pixels")

*   **1x** Personal Tricorder

#### [Lost and Found]( "Lost and Found")

*   **44x** [Pixels]( "Pixels")

*   **1x** Personal Tricorder

#### [Penguin Bay]( "Penguin Bay")

*   **44x** [Pixels]( "Pixels")

*   **1x** Personal Tricorder

#### [Tinkering Table]( "Tinkering Table")

*   **2x** [Silicon Board]( "Silicon Board")
*   **1x** [Glass]( "Glass")
*   **1x** [Tungsten Bar]( "Tungsten Bar")

*   **1x** Personal Tricorder

Used for
--------

#### [Tinkering Table]( "Tinkering Table")

*   **2x** [Electromagnet]( "Electromagnet")
*   **1x** [Advanced Plas

In [21]:
tricorder_4 = "can you tell me a few things that i can craft (along with its  required crafting amterials) with the tricorder?"
result_4 = rag_chain.invoke(tricorder_4)
print(result_4)


>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Getting_Started', 'Personal_Tricorder']
======= Doc 1 =======
[![](

How it looks (main tab of Tricorder).

[![](

The Quickbar (if you have either [Stardust Core Light]( or [Stardust Core]( mod) gives fast access to its primary and secondary features, but not its MM Config features.

How to obtain
-------------

#### [Lorewalker's Shop]( "Lorewalker's Shop")

*   **200x** [Pixels]( "Pixels")

*   **1x** Personal Tricorder

#### [Lost and Found]( "Lost and Found")

*   **44x** [Pixels]( "Pixels")

*   **1x** Personal Tricorder

#### [Penguin Bay]( "Penguin Bay")

*   **44x** [Pixels]( "Pixels")

*   **1x** Personal Tricorder

#### [Tinkering Table]( "Tinkering Table")

*   **2x** [Silicon Board]( "Silicon Board")
*   **1x** [Glass]( "Glass")
*   **1x** [Tungsten Bar]( "Tungsten Bar")

*   **1x** Personal Tricorder

Used for
--------

#### [Tinkering Table]( "Tinkering Table")

*   **2x** [Electromagnet]( "Electromagnet")
*   **1x** [Advance

In [22]:
# Stars
q5 = "what kinds of planets are in gentle stars?"
r5 = rag_chain.invoke(q5)
print(r5)


>>>> SOURCES <<<<< :
['Planets', 'Stars', 'Stars']
======= Doc 1 =======
Title: Planets - Frackin' Universe Wiki

URL Source: 

Published Time: 2023-08-30T05:32:18Z

Markdown Content:
These are all planets in FU and vanilla.

*   "Tier" is the same as "fauna threat". It will affect the strength of monsters, available ores (e.g. tier 3 [Titanium Ore]( "Titanium Ore") won't be on tier 2 planets) and tier of randomly generated weapons (if/when they drop from defeated enemies).
*   Light level only matters for [Solar Panels]( "Solar Panel"). Planets with "worst light level" of 0.75 or greater are perfect for solar power (4W from each Solar Panel). See also: [Weather Beacon]( "Weather Beacon").
*   Gravity 80 is "normal" (same as on the starting [Garden]( "Garden") planet). Some planets have fixed gravity (e.g. 55-80 for [Crystalline]( "Crystalline")). If gravity for some planet type is not listed, it means that gravity depends exclusively on the size of the planet.

Planet

Lowest tier

H

In [23]:
# Planets
q6 = "what locations can i find gelatinous planets?" # only takes sources from the Stars page (in Planets, it says Gentle Stars, Temperate Stars etc.)
r6 = rag_chain.invoke(q6) 
print(r6)


>>>> SOURCES <<<<< :
['Stars', 'Stars', 'Biomes']
======= Doc 1 =======
Frozen

[Aether World]( "Aether World"), [Dark Snow]( "Dark Snow"), [Dark Arctic]( "Dark Arctic"), [Dark Ice Waste]( "Dark Ice Waste"), [Dark Tundra]( "Dark Tundra"), [Penumbra]( "Penumbra"), [Shadow]( "Shadow")

Changes to Vanilla Stars\[[edit]( "Edit section: Changes to Vanilla Stars")\]
--------------------------------------------------------------------------------------------------------------------------------------------------

*   The threat level of [![]( "Gentle Star") [Gentle Star]( "Gentle Star (page does not exist)") systems is reduced from 3 to 2.
*    [![]( "Gentle Star") [Gentle Star]( "Gentle Star (page does not exist)") systems can contain planets of type [Crystalline]( "Crystalline"), [Rainforest]( "Rainforest"), [Gelatinous]( "Gelatinous"), [Bog]( "Bog"), [Primeval Forest]( "Primeval Forest") and [Fungal]( "Fungal").
*    [![]( "Temperate Star") [Temperate Star]( "Temperate Star (page does not 

In [24]:
# Planets
# q7 = "what's the fauna threat for gelatinous planets?"
q7 = "what's the highest tier for a normal volcaninc planet?"
r7 = rag_chain.invoke(q7)
print(r7)


>>>> SOURCES <<<<< :
['Planets', 'Stars', 'Planets']
======= Doc 1 =======
Title: Planets - Frackin' Universe Wiki

URL Source: 

Published Time: 2023-08-30T05:32:18Z

Markdown Content:
These are all planets in FU and vanilla.

*   "Tier" is the same as "fauna threat". It will affect the strength of monsters, available ores (e.g. tier 3 [Titanium Ore]( "Titanium Ore") won't be on tier 2 planets) and tier of randomly generated weapons (if/when they drop from defeated enemies).
*   Light level only matters for [Solar Panels]( "Solar Panel"). Planets with "worst light level" of 0.75 or greater are perfect for solar power (4W from each Solar Panel). See also: [Weather Beacon]( "Weather Beacon").
*   Gravity 80 is "normal" (same as on the starting [Garden]( "Garden") planet). Some planets have fixed gravity (e.g. 55-80 for [Crystalline]( "Crystalline")). If gravity for some planet type is not listed, it means that gravity depends exclusively on the size of the planet.

Planet

Lowest tier


In [25]:
# Biomes
q8 = "what's the reason to visit for Bog biomes?"
r8 = rag_chain.invoke(q8)
print(r8)


>>>> SOURCES <<<<< :
['Stars', 'Biomes', 'Stars']
======= Doc 1 =======
Frozen

[Aether World]( "Aether World"), [Dark Snow]( "Dark Snow"), [Dark Arctic]( "Dark Arctic"), [Dark Ice Waste]( "Dark Ice Waste"), [Dark Tundra]( "Dark Tundra"), [Penumbra]( "Penumbra"), [Shadow]( "Shadow")

Changes to Vanilla Stars\[[edit]( "Edit section: Changes to Vanilla Stars")\]
--------------------------------------------------------------------------------------------------------------------------------------------------

*   The threat level of [![]( "Gentle Star") [Gentle Star]( "Gentle Star (page does not exist)") systems is reduced from 3 to 2.
*    [![]( "Gentle Star") [Gentle Star]( "Gentle Star (page does not exist)") systems can contain planets of type [Crystalline]( "Crystalline"), [Rainforest]( "Rainforest"), [Gelatinous]( "Gelatinous"), [Bog]( "Bog"), [Primeval Forest]( "Primeval Forest") and [Fungal]( "Fungal").
*    [![]( "Temperate Star") [Temperate Star]( "Temperate Star (page does not 

#### UI

In [27]:
import gradio as gr
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
# from langchain_community.chat_models import ChatOllama
from langchain.llms import OpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter

gr_embedding = embedding
gr_documents = doc_splits
gr_llm = model_local

def process_input(urls, question):
    # model_local = ChatOllama(model="mistral")
    
    # Convert string of URLs to list
    urls_list = urls.split("\n")
    # docs = [WebBaseLoader(url).load() for url in urls_list]
    # docs_list = [item for sublist in docs for item in sublist]
    docs_list = [scrape_jina_ai_2(url) for url in urls_list] # scrape using Jina AI's reader
    
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
    # text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
    doc_splits = text_splitter.split_documents(docs_list)

    # vectorstore = Chroma.from_documents(
    #     documents=doc_splits,
    #     collection_name="rag-chroma",
    #     embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
    # )
    
    vectorstore = Chroma.from_documents(
        documents=gr_documents,
        collection_name="rag-chroma",
        embedding=gr_embedding,
    )
    
    # retriever = vectorstore.as_retriever()
    retriever = vectorstore.as_retriever(search_kwargs={"k":4})

    # after_rag_template = """Answer the question based only on the following context (If the information is not in the context, say you don't have that information):
    # {context}
    # Question: {question}
    # """
    llama_template = """
                <|begin_of_text|><|start_header_id|>system<|end_header_id|>
                You are a pedantic but knowledgeable, efficient and direct AI assistant for Frackin' Universe Website. Provide concise answers focusing on key information. Offer tactful suggestions to solve the user's question. 
                Answer the *question* based only on the following *context*. If the information is not in the *context*, say that you don't have the informmation.
                Focus on the MAIN contents of the given webpage in the *context*, ignoring the periphery of the website i.e., navigation bars, headers, footers, social media etc. 
                It is **CRITICAL** that you thoroughly digest the given *context* to answer the user's *question*.
                Context: {context}
                <|eot_id|>
                <|start_header_id|>user<|end_header_id|>
                {question}
                <|eot_id|>
                <|start_header_id|>assistant<|end_header_id|>
                """
    
    after_rag_prompt = ChatPromptTemplate.from_template(llama_template)
    after_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        # | model_local
        | gr_llm
        | StrOutputParser()
    )
    return after_rag_chain.invoke(question)

# Define Gradio interface
iface = gr.Interface(fn=process_input,
                     inputs=[gr.Textbox(label="Enter URLs separated by new lines"), gr.Textbox(label="Question")],
                     outputs="text",
                     title="Webpage Query",
                     description="Enter URLs and a question to query the documents.")
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
